In [6]:
from sklearn.pipeline import Pipeline, FeatureUnion
import sklearn.decomposition as decomposition
from sklearn.decomposition import SparsePCA
import sklearn.preprocessing as preprocessing
from sklearn.feature_selection import SelectKBest
import pandas as pd
import sklearn.preprocessing as pp
import numpy as np
import sklearn.naive_bayes as naive_bayes
import sklearn.cross_validation as cross_validation
import Classifier as cnb
from sklearn.metrics import accuracy_score

# Loading the Data

In [24]:
X = pd.read_csv('../assignment1_2016S1/training_data.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]
y = pd.read_csv('../assignment1_2016S1/training_labels.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]

In [27]:
X_scaled = X.applymap(lambda x: 1 if x > 0 else 0)

# Feature Engineering / Pre-Processing

In [41]:
svd = decomposition.TruncatedSVD(n_components=700,random_state=42)

In [39]:
pca = decomposition.PCA(n_components=0.90)

In [46]:
count = []
for name,series in X_scaled.iteritems():
    if(series.sum() < 500):
        count += [name]
len(count)

13048

In [47]:
temp = pd.DataFrame(X[count])
temp[13627] = temp.sum(axis=1)
temp.shape

(20104, 13049)

# Naive Bayes (Gaussian and Multinomial)

In [49]:
X_train,X_test,y_train,y_test = cross_validation.train_test_split(temp, y, test_size=0.33, random_state=42)

nb = naive_bayes.GaussianNB()
mnb = naive_bayes.MultinomialNB()

raw_pipe = Pipeline([("naive_bayes",nb)])
pca_pipe = Pipeline([("pca",pca),("naive_bayes", nb)])
svd_pipe = Pipeline([("svd",svd),("naive_bayes",nb)])
raw_pipe_mnb = Pipeline([("naive_bayes",mnb)])
pipes = [raw_pipe,pca_pipe,svd_pipe,raw_pipe_mnb]
for pipeline in pipes:
    print(pipeline.fit(X_train,y_train.as_matrix().ravel()).score(X_test,y_test))

0.429238884702
0.188997739261
0.0795779954785
0.503391107762


# Multinomial Naive Bayes

# Decision Tree

# SVM - Linear Kernel

# Random Forest

In [3]:
def build_model(data):
    """
    This function builds tree from Pandas Data Frame with last column as the dependent feature
    """
    attributes = list(data.columns.values)
    target = attributes[-1]
    classes = nb.get_attribute_values(data,target)
    class_frequency = data[target].value_counts().sort_index()
    betak = [2]*len(classes)
    betam = (100/class_frequency + 1).astype(int)
    return nb.NaiveBayesMAP(data,target,betak,betam)

In [11]:
data =  pd.read_csv('../zoo.csv');
    del data['animalname']
    print("Data Read and Loaded")
    X_train, X_valid =  cross_validation.train_test_split(data, test_size=0.33, random_state=42)
    model = build_model(X_train)
    actuals = X_valid.type
    del X_valid['type']
    preds = nb.predict(model,X_valid)
    print("Classification Accuracy is " + str(accuracy_score(actuals,preds)))

Data Read and Loaded


84    1
55    1
66    1
67    1
45    1
39    6
22    1
44    1
10    1
0     1
18    4
30    6
97    6
33    2
77    7
4     1
93    1
78    2
12    4
31    1
76    3
89    5
26    5
42    6
70    1
15    7
40    6
72    7
9     1
96    1
11    2
91    3
64    1
28    1
Name: type, dtype: int64